# Обучение рекуррентной нейронной сети для генерации текстов

В этом ноутбуке мы обучим рекуррентную нейросеть небольшого размера на первом томе романа "Война и мир" Л. Н. Толстого. Такая нейронная сеть будет уметь генерировать продолжения фраз на русском языке. В задании Вы попробуете выполнять такие предсказания. 

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace

### Гиперпараметры

In [5]:
flags = Namespace(
    train_file='VoinaIMir.txt', # файл с данными
    seq_size=32, # максимальная длина текста (в словах)
    batch_size=16, # размер мини-батча
    embedding_size=64, # число элементов в векторных представлениях слов
    lstm_size=64, # число нейронов в рекуррентном слое
    gradients_norm=5 # ограничение на норму градиентов (улучшает обучение)
)


### Загрузка данных

Функция get_data_from_file выполняет следующие действия:
* чтение файла с данными
* сборка словаря (всех возможных слов) и сортировка по частоте
* предобработка данных (замена слов на числа - индексы в словаре)

Функция get_batches генерирует мини-батчи для обучения.

In [6]:
def get_data_from_file(train_file, batch_size, seq_size):
    # чтение файла с данными
    with open(train_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text = text.split()

    # сборка словаря (всех возможных слов) и сортировка по частоте
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Размер словаря', n_vocab)

    # предобработка данных
    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text


def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

Загрузим данные:

In [7]:
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = \
    get_data_from_file(
    flags.train_file,\
    flags.batch_size, \
    flags.seq_size)

Размер словаря 22782


In [14]:
n_vocab

22782

Посмотрим на 20 самых частых слов:

In [9]:
for idx in range(20):
    print(int_to_vocab[idx])

,
.
и
—
в
не
на
что
он
с
как
!
к
его
?
сказал
я
было
это
;


Найдем номер слова "Наташа":

In [10]:
vocab_to_int["Наташа"]

252

А вот как выглядит одна строчка обработанных данных (это номера слов в словаре):

In [13]:
in_text[0, :10]

array([8635, 5267, 8636, 8637, 8638, 3804,   34, 8639,    2,  965])

### Сборка нейросети

In [4]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        # слой векторных представлений (эмбеддингов)
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        # рекуррентный слой
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        # полносвязный слой для предсказания следующего слова
        self.dense = nn.Linear(lstm_size, n_vocab)

    def forward(self, x, prev_state):
        # x: (16, 32): 16 объектов, 32 слова в каждом
        embed = self.embedding(x) # (16, 32, 64): 64 - размер векторного представления
        output, state = self.lstm(embed, prev_state) # (16, 32, 64)
        logits = self.dense(output) # (16, 32, 22782), 22782 - число слов в словаре

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

Функция для задания оптимизируемого критерия и оптимизитора:

In [5]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

Функция для выполнения предсказания (закончить данный текст):

In [171]:
def predict(device, net, n_vocab, vocab_to_int, int_to_vocab,
             words="Зачем мне", top_k=5, length=100):
    words = words.split()
    
    net.eval()

    # обработка данных слов
    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

    # предсказание первого следующего слова (случайный выбор из top_k слов)
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])

    # повторение процедуры генерации length раз
    for _ in range(length):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))
    return ' '.join(words)

### Обучение нейросети

In [7]:
# устройство, на котором будут храниться модель и мини-батчи
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# создаем конкретный экземпляр нейросети
net = RNNModule(n_vocab, flags.seq_size,
                flags.embedding_size, flags.lstm_size)
# переносим нейросеть на устройство
net = net.to(device)

# оптимизируемый критерий, оптимизатор
criterion, optimizer = get_loss_and_train_op(net, 0.01)

iteration = 0

# цикл по эпохам
for e in range(200):
    batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
    state_h, state_c = net.zero_state(flags.batch_size)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    # цикл по мини-батчам
    for x, y in batches:
        iteration += 1
        net.train()

        optimizer.zero_grad()
        
        # переносим мини-батч на устройство
        x = torch.tensor(x).to(device)
        y = torch.tensor(y).to(device)

        # проход вперед - вычиссление предсказаний
        logits, (state_h, state_c) = net(x, (state_h, state_c))
        
        # вычисление критерия
        loss = criterion(logits.transpose(1, 2), y)

        loss_value = loss.item()

        # проход назад - вычисление производных
        loss.backward()

        state_h = state_h.detach()
        state_c = state_c.detach()

        _ = torch.nn.utils.clip_grad_norm_(
            net.parameters(), flags.gradients_norm)

        # обновление параметров модели
        optimizer.step()

        # иногда печатаем промежуточные результаты
        if iteration % 1000 == 0:
            print('Epoch: {}/{}'.format(e, 200),
                  'Iteration: {}'.format(iteration),
                  'Loss: {}'.format(loss_value))
            predict(device, net, n_vocab,
                    vocab_to_int, int_to_vocab, top_k=5)

Vocabulary size 22782
Epoch: 3/200 Iteration: 1000 Loss: 4.871341228485107
Зачем мне самую плохую , я вам скажу ! … Я рад , как и — кричал четвертый , что он , как и не только в тумане , — сказала она , но с большим роты , что в то , — сказал Кутузов . « Но он сам , и он видел его высочество , но , что это не мог быть иначе и , но в самом изящном или к себе . — Я знаю . — Да ! » думал князь Василий в тумане . В одной день то на него , — продолжал он , и не было
Epoch: 7/200 Iteration: 2000 Loss: 3.65338397026062
Зачем мне удовлетворение… ] что ? А ? ! — обратился Анатоль в то же место занимают и изящества . В то мгновение куда что она затевала между Шлапаницем в отчаянии и не может не понравиться накануне , и в лесу , и более и крик . « Ежели вы отправляете , я вас . — Что греха таить не будет . — Вы знаете за границей . [ Этот послал к petite est un… ] желает назначения Жюли ; но не позволю нему ! что не может видеть и не так на себе нынче ? … Я думаю . И она не
Epoch: 10/200 Iteratio

In [20]:
net.cpu()
save = {"net":net.state_dict(), \
        "n_vocab":n_vocab, \
        "vocab_to_int":vocab_to_int,\
        "int_to_vocab":int_to_vocab}
torch.save(save, "net.dict")

### Пробуем делать предсказания

Функция predict принимает на вход следующие аргументы:
* device: на каком устройстве выполняются предсказания - см. предыдущую ячейку
* net: обученная нейросеть - см. предыдущую ячейку
* n_vocab: число слов в словаре - см. раздел Подготовка данных
* vocab_to_int: словарь, возвращающий номер слова - см. раздел Подготовка данных
* int_to_vocab: словарь, возвращающий слова по их номеру - см. раздел Подготовка данных
* (!) my_input: строка, которую нужно продолжить
* (!) top_k: из скольки слов с наибольшими предсказанными вероятностями выбирать следующее слово 
* (!) length: сколько слов генерировать

In [24]:
my_input = "Я"
predict(device, net, n_vocab, vocab_to_int, int_to_vocab, \
        my_input, top_k=1, length=20)

Я вас люблю , — сказал он , — я повторить не могу , — невозможно . — Это-то мы и посмотрим


In [25]:
my_input = "Завтра поедем выбирать новую шубу"
predict(device, net, n_vocab,
                    vocab_to_int, int_to_vocab, my_input, top_k=1,\
       length=10)

Завтра поедем выбирать новую шубу , — сказала старая графиня , проходя через залу и улыбнулась


In [26]:
my_input = "Завтра поедем выбирать новую машину"
predict(device, net, n_vocab,
                    vocab_to_int, int_to_vocab, my_input, top_k=1,\
       length=10)

Завтра поедем выбирать новую машину . Но князь Василий ? — спросила графиня . — Ах


In [27]:
my_input = "Завтра поедем выбирать новое платье"
predict(device, net, n_vocab,
                    vocab_to_int, int_to_vocab, my_input, top_k=1,\
       length=10)

Завтра поедем выбирать новое платье . Ее находят прекрасною , как бы счастливая их . —


In [191]:
my_input = "Василий  Денисов ,  друг"
result = predict(device, net, n_vocab,
                    vocab_to_int, int_to_vocab, my_input, top_k=1,\
       length=20)
result

Василий Денисов , друг сердечный , как вы ко всем министрам , ко всему . Если Богу угодно будет испытать тебя . Вот так бедны


'Василий Денисов , друг сердечный , как вы ко всем министрам , ко всему . Если Богу угодно будет испытать тебя . Вот так бедны'

Попробуйте генерировать слои предсказания в ноутбуке с заданием!